In [1]:
from __future__ import division
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"#-----specify device gpu or cpu
import cv2
import numpy as np
import sys
import pickle
from optparse import OptionParser
import time
from keras_frcnn import config
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from keras_frcnn.wwk_parser import get_data

Using TensorFlow backend.


In [2]:
dir_path='/home/zoro/Desktop/nmumg_data'
main_path=dir_path+'/train_test_data'
data_path=dir_path+'/train_test_data/test'
img_path=dir_path+'/frcnn_predict' #-----for prediction-------
path_output=dir_path+'/output'

In [3]:
C=config.Config()
if C.network == 'resnet50':
	import keras_frcnn.resnet as nn
elif C.network == 'vgg':
	import keras_frcnn.vgg as nn

print C.network
# turn off any data augmentation at test timeuse_horizontal_flips
C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False


vgg


In [4]:
#--------note wwk---------- ratio is C.im_size/train_img_size
def format_img_size(img, C):
	""" formats the image size based on config """
	(height,width,_) = img.shape
	    
	ratio=float(C.im_size)/float(C.train_img_size)
	new_width = int(ratio * width)
	new_height = int(ratio*height)
	print img.shape
	img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
	img = np.reshape(img,(new_width,new_height,1))    
	return img, ratio	

def format_img_channels(img, C):
	""" formats the image channels based on config """
# 	img = img[:, :, (2, 1, 0)]
	img = img.astype(np.float32)
# 	img[:, :, 0] -= C.img_channel_mean[0]
# 	img[:, :, 1] -= C.img_channel_mean[1]
# 	img[:, :, 2] -= C.img_channel_mean[2]
# 	img /= C.img_scaling_factor
	img[:, :, 0] -= C.img_mean
#	img = np.transpose(img, (2, 0, 1))
	img = np.expand_dims(img, axis=0)
	return img

def format_img(img, C):
	""" formats an image for model prediction based on config """
	img, ratio = format_img_size(img, C)
	img = format_img_channels(img, C)
	return img, ratio

# Method to transform the coordinates of the bounding box to its original size
def get_real_coordinates(ratio, x1, y1, x2, y2):

	real_x1 = int(round(x1 // ratio))
	real_y1 = int(round(y1 // ratio))
	real_x2 = int(round(x2 // ratio))
	real_y2 = int(round(y2 // ratio))

	return (real_x1, real_y1, real_x2 ,real_y2)

In [5]:
all_imgs, classes_count, class_mapping = get_data(main_path,data_path)

if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping
print class_mapping

Parsing annotation files
{'cell': 0, 'bg': 1}


In [6]:
if 'bg' not in class_mapping:#background
	class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}
#C.num_rois = int(options.num_rois)

{0: 'cell', 1: 'bg'}


In [7]:
if C.network == 'resnet50':
	num_features = 1024
elif C.network == 'vgg':
	num_features = 512

if K.image_dim_ordering() == 'th':
	input_shape_img = (1, None, None)
	input_shape_features = (num_features, None, None)
else:
	input_shape_img = (None, None, 1)
	input_shape_features = (None, None, num_features)

print input_shape_img

(None, None, 1)


In [9]:
img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=input_shape_features)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)

model_rpn = Model(img_input, rpn_layers)
model_classifier_only = Model([feature_map_input, roi_input], classifier)

model_classifier = Model([feature_map_input, roi_input], classifier)

print('Loading weights from {}'.format(C.model_path))
model_rpn.load_weights(C.model_path, by_name=True)
model_classifier.load_weights(C.model_path, by_name=True)

model_rpn.compile(optimizer='sgd', loss='mse')
model_classifier.compile(optimizer='sgd', loss='mse')

Loading weights from model_frcnn_nmumg_ellipse.vgg.hdf5


In [ ]:
all_imgs = []

classes = {}
all_box=[]

bbox_threshold = 0.5#---note wwk--- probability threshold for classification

overlap_thresh=0.2# for non_max_suppression_fast

visualise = True

for idx, img_name in enumerate(sorted(os.listdir(img_path))):
	if not img_name.lower().endswith(('.tif', '.tiff')):
		continue
	print(img_name)
	st = time.time()
	filepath = os.path.join(img_path,img_name)

	img = cv2.imread(filepath,cv2.COLOR_BGR2GRAY)
	#img = bg_correction(img,ordr=1)#-----------backgroud correction--------------
	img=np.reshape(img,(img.shape[0],img.shape[1],1))
    
	#----note wwk----we train on real size image. cell in large images and the train images have the same size
	#----so when make prediction on images whose size is quite different from the training size, should not resize
	X, ratio = format_img(img, C)
# 	if K.image_dim_ordering() == 'tf':
# 		X = np.transpose(X, (0, 2, 3, 1))
# 	X=np.expand_dims(img, axis=0)#this is the alternative of X,ratio=format_img(img,C)
# 	ratio=1 
    
	# get the feature maps and output from the RPN
	[Y1, Y2, F] = model_rpn.predict(X)
	
	#----note wwk--- R shape(:,4)  x1,y1,x2,y2 of each anchor box   
	R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)

	# convert from (x1,y1,x2,y2) to (x,y,w,h)
	R[:, 2] -= R[:, 0]
	R[:, 3] -= R[:, 1]

	# apply the spatial pyramid pooling to the proposed regions
	bboxes = {}
	probs = {}

	for jk in range(R.shape[0]//C.num_rois + 1): # ---note wwk---calculate C.num_rois at the same time
		ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
		#print ROIs.shape        
		if ROIs.shape[1] == 0:
			break

		if jk == R.shape[0]//C.num_rois:#if jk==R.shape[0]//C.num_rois, ROIs.shape is not (1,x,4),x<C.num_rois
			#pad R
			curr_shape = ROIs.shape
			#print curr_shape            
			target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
			ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
			ROIs_padded[:, :curr_shape[1], :] = ROIs
			ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
			ROIs = ROIs_padded

		[P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

		#print P_cls.shape
		for ii in range(P_cls.shape[1]):
			#----note wwk--- if <threshold or the largest probability is background       
			if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
				continue

			cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

			if cls_name not in bboxes:
				bboxes[cls_name] = []
				probs[cls_name] = []

			(x, y, w, h) = ROIs[0, ii, :]

			cls_num = np.argmax(P_cls[0, ii, :])
			try:
				(tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
				tx /= C.classifier_regr_std[0]
				ty /= C.classifier_regr_std[1]
				tw /= C.classifier_regr_std[2]
				th /= C.classifier_regr_std[3]
				x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
			except:
				pass
			bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
			probs[cls_name].append(np.max(P_cls[0, ii, :]))
	print type(bboxes)
	all_dets = []

	for key in bboxes:
		print(key)        
		bbox = np.array(bboxes[key])        
		img=np.reshape(img,(img.shape[0],img.shape[1]))
		fig, ax = plt.subplots(figsize=(10, 6))
		ax.imshow(img)
		new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=overlap_thresh)
		real_boxes=np.zeros(new_boxes.shape)
		for jk in range(new_boxes.shape[0]):
			(x1, y1, x2, y2) = new_boxes[jk,:]

			(real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)
			print real_x1, real_y1, real_x2, real_y2
			real_boxes[jk,:]=real_x1,real_y1,real_x2,real_y2
# 			real_boxes[jk,1]=real_y1 
# 			real_boxes[jk,2]=real_x2
# 			real_boxes[jk,3]=real_y2
#			cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)


			rect = mpatches.Rectangle((real_x1,real_y1),real_x2-real_x1, real_y2-real_y1,fill = False, edgecolor = 'red', linewidth = 2)
			ax.add_patch(rect)
			plt.axis('off')        
			plt.savefig(path_output+'/box_predict.tif',bbox_inches='tight',format='tif',dpi=300)
            
			textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
			all_dets.append((key,100*new_probs[jk]))
		plt.show()       
		real_boxes=real_boxes.astype(int)
		all_box.append(real_boxes)
	print('Elapsed time = {}'.format(time.time() - st))
	print(all_dets)



reg_out_exact0.tif
(384, 384, 1)


In [19]:
import pickle
with open(path_output+"/bndbox.txt", "wb") as fp: #pickling
    pickle.dump(all_box, fp)    
with open(path_output+"/bndbox.txt", "rb") as fp: # Unpickling
    b = pickle.load(fp)
print(b)

[array([[368, 520, 424, 568],
       [520, 216, 560, 256],
       [360, 448, 416, 504],
       [448, 464, 504, 504],
       [536, 384, 576, 432],
       [496, 504, 568, 552],
       [352, 376, 400, 416],
       [520, 280, 576, 344],
       [416, 344, 480, 392],
       [240, 320, 296, 384],
       [544,  56, 584, 104],
       [168, 576, 208, 632],
       [296, 432, 336, 472],
       [336, 592, 392, 640],
       [472, 312, 528, 384],
       [576, 264, 608, 312],
       [440, 144, 504, 192],
       [488,  72, 528, 128],
       [416, 224, 496, 312],
       [232, 560, 272, 608],
       [272, 520, 320, 576],
       [336, 320, 384, 360],
       [528, 128, 616, 192],
       [224, 384, 272, 448],
       [376, 168, 432, 216],
       [312, 488, 352, 536],
       [192, 424, 232, 488],
       [592, 320, 640, 360],
       [224, 464, 280, 528],
       [432, 536, 512, 592],
       [400, 408, 480, 448],
       [168, 512, 208, 560],
       [304, 208, 392, 256],
       [520, 448, 584, 496],
       [592, 